# Perdido: Python library for geoparsing and geocoding French texts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ludovicmoncla/perdido/blob/main/notebooks/perdido-geoparser-GeoExT-ECIR23.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ludovicmoncla/perdido/main?filepath=notebooks/perdido-geoparser-GeoExT-ECIR23.ipynb)

This notebook is proposed by [Ludovic Moncla](https://ludovicmoncla.github.io) (Univ Lyon, INSA Lyon, CNRS, UCBL, LIRIS, UMR 5205, F-69621) and [Mauro Gaio](https://lma-umr5142.univ-pau.fr/fr/organisation/membres/cv_-mgaio-fr.html) (Université de Pau et des Pays de l'Adour, CNRS, LMAP, UMR 5142) and was presented at the First International Workshop on Geographic Information Extraction from Texts (GeoExT'23) ([https://geo-ext.github.io](https://geo-ext.github.io)) held at the [ECIR 2023](https://ecir2023.org) conference in Dublin.


## Cite this work
> Moncla, L. and Gaio, M. (2023). Perdido: Python library for geoparsing and geocoding French texts. In proceedings of the First International Workshop on Geographic Information Extraction from Texts (GeoExT'23), ECIR Conference, Dublin, Ireland.


## 1. Overview

In this tutorial, we'll learn about a few different things:

- Use the `Perdido Geoparser` library for geoparsing French texts (geotagging + geocoding)
  - Display geotagging results
  - Map geocoding results
  - Save the results in different formats (csv, dataframe, ...)
  - Illustrate the problem of disambiguation of toponyms
- Compare `Perdido` NER results with `spaCy` and `Stanza` (python libraries)
    

## 2. Setting up the environment


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 3.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter la cellule suivante :

In [ ]:
! pip install perdido==0.1.36
! pip install display-xml==0.1.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 3. Perdido Geoparser

`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le **geotagging** et le **geocoding**. 

Le geotagging est similaire à la tâche de **reconnaissance des entités nommées** avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).

Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour le geoparsing.

### 3.1 Importer la librairie

In [ ]:
from perdido.geoparser import Geoparser

### 3.2 Executer le geoparser

In [ ]:
text = "J'aimerais vous proposer un rendez-vous, dans un lieu tenu secret à Lyon, "
text += "proche de la place Bellecour, de la place des Célestins, "
text += "au sud de la fontaine des Jacobins et près du pont Bonaparte."

In [ ]:
geoparser = Geoparser()
doc = geoparser(text)

### 3.3 Visualiser les résultats

* Visualiser les attributs des tokens :

In [ ]:
for token in doc:
    print(f'{token.text}\tlemma: {token.lemma}\tpos: {token.pos}')

* Format IOB :

In [ ]:
for token in doc:
    print(token.iob_format())

* Format IOB-TSV :

In [ ]:
for token in doc:
    print(token.tsv_format())

* Afficher la sortie XML-TEI. Voir la référence ci-après pour une description du schéma de balisage :

> Moncla, L., & Gaio, M. (2015). A multi-layer markup language for geospatial semantic annotations. Proceedings of the 9th Workshop on Geographic Information Retrieval, 1–10. Paris, France. [https://doi.org/10.1145/2837689.2837700](https://doi.org/10.1145/2837689.2837700)

In [ ]:
print(doc.tei)

* Utilisation de la librairie [display_xml](https://github.com/mpacer/display_xml) pour affichage du XML avec coloration syntaxique :

In [ ]:
from display_xml import XML

XML(doc.tei, style='lovelace')

* Afficher la sortie GeoJSON :

In [ ]:
doc.geojson

* Afficher la liste des entités nommées :

In [ ]:
for entity in doc.named_entities:
    print(f'entity: {entity.text}\ttag: {entity.tag}')
    if entity.tag == 'place':
        for t in entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')

* Afficher la liste des entités nommées étendues :

In [ ]:
for nested_entity in doc.nested_named_entities:
    print(f'entity: {nested_entity.text}\ttag: {nested_entity.tag}')
    if nested_entity.tag == 'place':
        for t in nested_entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')


* Affichage graphique des résultats avec la librairie [spacy](https://spacy.io/usage/visualizers) :

In [ ]:
from spacy import displacy

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

In [ ]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

* Affichage de la carte générée à partir du GeoJSON :

In [ ]:
doc.get_folium_map()

### 3.4 Exporter les résultats

* Enregistrer les résultats au format XML-TEI :

In [ ]:
doc.to_xml('filename.xml')

* Enregistrer les résultats au format GeoJSON :

In [ ]:
doc.to_geojson('filename.geojson')

* Enregistrer les résultats au format IOB-TSV :

In [ ]:
doc.to_iob('filename.tsv')

* Enregistrer les résultats au format CSV :

In [ ]:
doc.to_csv('filename.csv')

### 3.5 Paramétrage

La librairie est disponible en 2 versions : une version `Standard` et une version `Encyclopedie` spécialement adaptée pour les articles encyclopédiques. L'étape de géocoding est fortement paramétrable, en particulier afin de filtrer les résultats provenants des ressources géographiques (dans le but de limiter les ambiguïtés).

#### 3.5.1 Paramétrage du geotagging

* Paramétrer la version des règles d'annotation utilisée pour la reconnaissance des entités nommées :

    * `Standard` (par défaut): règles développées initialement pour le traitement de descriptions de randonnées
    * `Encyclopedie` : règles adaptées pour le traitement d'article encyclopédique

In [ ]:
text = "ARQUES, (Géog.) petite ville de France, en Normandie, "
text += "au pays de Caux, sur la petite riviere d'Arques. Long. 18. 50. lat. 49. 54."

geoparser = Geoparser(version="Encyclopedie")
doc = geoparser(text)

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

#### 3.5.2 Paramétrage du geocoding

* Pour les exemples suivants nous allons utiliser un extrait d'une description de randonnée :

In [ ]:
text  = "Départ du parking du Cleyat. Suivre le chemin qui monte dans la forêt en direction du col de La Ruchère. "
text += "Au col, monter au petit Som par la voie la plus raide avec le passage du goulot qui arrive directement au petit Som: glissant et un peu d'escalade ! "
text += "Arrivé au petit Som, à la croix: vue 360. Descente au col de Léchaud puis col de Bovinant. Rejoindre le habert de Bovinant. "
text += "Tentative de montée au grand Som mais premier goulot très glissant: demi-tour. Descendre en direction du col de la Ruchère par le pas du loup. "
text += "Passage délicat au pas du loup. Rejoindre le col de la Ruchère."

* On utilisera également la trace GPS pour l'afficher sur la carte et pouvoir comparer avec les lieux géocodés :

In [ ]:
import shapely.wkt

# Créer l'objet géométrie à partir d'un string
gpx_geom = shapely.wkt.loads("MULTILINESTRING ((5.798105 45.39974, 5.797635 45.39895, 5.797609 45.397997, 5.797635 45.397279, 5.797348 45.396596, 5.797061 45.396003, 5.796905 45.395266, 5.797192 45.393937, 5.797296 45.392949, 5.797296 45.392033, 5.797218 45.391116, 5.796513 45.390667, 5.796226 45.390218, 5.795939 45.389409, 5.795391 45.388924, 5.79479 45.388619, 5.794843 45.387685, 5.795234 45.387271, 5.7962 45.387038, 5.796591 45.38693, 5.797818 45.386463, 5.798653 45.386481, 5.799671 45.386014, 5.799854 45.385744, 5.800115 45.385475, 5.800376 45.385421, 5.800637 45.385852, 5.800898 45.385798, 5.800924 45.385547, 5.801289 45.385744, 5.801838 45.385439, 5.801916 45.385798, 5.803665 45.386337, 5.804134 45.386122, 5.804239 45.386247, 5.804187 45.386625, 5.804369 45.386876, 5.804787 45.386912, 5.805022 45.386876, 5.805205 45.386966, 5.805126 45.386768, 5.805596 45.386858, 5.805518 45.386768, 5.805674 45.386625, 5.805779 45.386804, 5.805883 45.38702, 5.805674 45.387379, 5.805648 45.387793, 5.805231 45.388062, 5.805805 45.387739, 5.806431 45.387577, 5.806901 45.387182, 5.807736 45.387236, 5.808493 45.387218, 5.808859 45.387218, 5.810216 45.386643, 5.810529 45.386481, 5.811651 45.385834, 5.811886 45.385726, 5.811912 45.385439, 5.812069 45.385349, 5.812199 45.3849, 5.812304 45.384846, 5.812408 45.384559, 5.812434 45.384415, 5.812904 45.384451, 5.813139 45.384379, 5.813583 45.384559, 5.814261 45.384648, 5.81447 45.384361, 5.814548 45.383966, 5.814235 45.383409, 5.813948 45.382726, 5.813844 45.382439, 5.813896 45.382025, 5.81434 45.381882, 5.81434 45.381864, 5.813844 45.382043, 5.813818 45.382474, 5.814601 45.38393, 5.814522 45.384415, 5.814105 45.384271, 5.813531 45.383678, 5.813165 45.383301, 5.812826 45.382798, 5.81233 45.382259, 5.812121 45.382259, 5.811964 45.381953, 5.811521 45.3819, 5.811234 45.382133, 5.810947 45.382133, 5.811208 45.38181, 5.811208 45.381558, 5.810999 45.38145, 5.810581 45.381361, 5.810529 45.381846, 5.810581 45.381935, 5.810425 45.382115, 5.809824 45.382474, 5.809563 45.383139, 5.80925 45.383337, 5.808598 45.38357, 5.807997 45.383534, 5.80758 45.383984, 5.806797 45.383229, 5.806457 45.382942, 5.80617 45.383283, 5.806092 45.383894, 5.805439 45.384415, 5.804474 45.384864, 5.803665 45.385349, 5.803116 45.385672, 5.80236 45.385744, 5.802125 45.385726, 5.801942 45.385888, 5.801916 45.385816, 5.801916 45.385421, 5.801237 45.385726, 5.800898 45.385583, 5.800794 45.38587, 5.800663 45.38587, 5.800272 45.385457, 5.800115 45.385439, 5.799671 45.385978, 5.798706 45.386463, 5.797714 45.386481, 5.796383 45.386984, 5.79526 45.387271, 5.794869 45.387739, 5.794738 45.388583, 5.795443 45.388924, 5.795965 45.389374, 5.796252 45.390236, 5.797244 45.39108, 5.797244 45.392374, 5.797296 45.393218, 5.797113 45.394045, 5.796878 45.395266, 5.797087 45.396147, 5.797583 45.397027, 5.797557 45.397512, 5.797609 45.398932, 5.798184 45.39974, 5.798053 45.399668))")

* Paramétrer la ou les ressources utilisées (gazetier) : 

    * `nominatim` (par défaut): [OpenStreetMap](https://www.openstreetmap.org)
    * `ign` : [GeoPortail](https://www.geoportail.gouv.fr)
    * `geonames` : [Geonames](http://www.geonames.org)
    * `whg`: [World Historical Gazetteer](https://whgazetteer.org)
    * `pleiades`: [Pleiades](https://pleiades.stoa.org)

In [ ]:
geoparser = Geoparser(sources=['ign'])
doc = geoparser(text)
doc.get_folium_map()

* L'objet `Perdido` a un attribut `geometry_layer` qui peut prendre la valeur d'une couche de géométrie à afficher sur la carte. Dans notre cas, on ajoute la géométrie de la trace GPS de la randonnées :

In [ ]:
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* On refait le même traitement mais avec OpenStreetMap:

In [ ]:
geoparser = Geoparser(sources=['nominatim'])
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Paramétrer le mode du geocoding (0: geocoder seulement l'EN, 1 (default): seulement l'ENE et les EN hors ENE, 2 géocoder à la fois les ENE et les EN imbriquées) :

In [ ]:
geoparser = Geoparser(geocoding_mode=0)
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Paramétrer le nombre de résultats retournés pour chaque toponyme (par ressource), 1 par défaut :

In [ ]:
geoparser = Geoparser(max_rows=10)
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Filtrer les résultats par pays (code pays) :

In [ ]:
geoparser = Geoparser(max_rows=10, country_code = 'fr')
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

* Filtrer les résultats selon une zone géographique (bounding box: `east`,`south`,`west`,`north`) :

In [ ]:
geoparser = Geoparser(max_rows=10, bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403])
doc = geoparser(text)
doc.geometry_layer = gpx_geom
doc.get_folium_map()

### 3.6 Désambiguïsation

La librairie Perdido est toujours en cours de développement et d'amélioration dans le cadre de différents projet de recherche ([ANR CHOCUAS](), [GEODE]()), à l'heure actuelle une seule méthode de désambiguïsation automatique est disponible. Il s'agit d'une méthode de filtrage par clustering.


#### 3.6.1 Clustering par densité spatiale

Le principe est de regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*) et de selectionner le cluster qui contient le plus d'entités distinctes.

> Moncla, L., Renteria-Agualimpia, W., Nogueras-Iso, J., & Gaio, M. (2014). Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus. Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, 183–192. Dallas, TX, USA: ACM. [https://doi.org/10.1145/2666310.2666386](https://doi.org/10.1145/2666310.2666386)

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 

Cette stratégie est adaptée pour des textes où les différents lieux cités sont supposés être localisés à proximité les uns des autres (ex: descriptions de randonnées).

In [ ]:
geoparser = Geoparser(max_rows=10)
doc = geoparser(text)

doc.cluster_disambiguation()

doc.geometry_layer = gpx_geom
doc.get_folium_map()

## 4. Perdido Geocoder

En plus de la classe Geoparser, la librairie Perdido propose aussi la classe Geocoder. Cette classe permet de geocoder un ou plusieurs toponymes. Les paramètres sont les mêmes que ceux utilisés avec le geoparser pour configuer l'étape de geocoding :
* sources
* max_row
* country_code
* bbox

L'objet retourné est de type Perdido comme pour le Geoparser ce qui permet d'avoir accès aux mêmes attributs et méthodes que précédemment.

### 4.1 Importer la librairie

In [ ]:
from perdido.geocoder import Geocoder

### 4.2 Executer le geocoder

* Instancier le geocoder :

In [ ]:
geocoder = Geocoder()

* Geocoder un nom de lieu :

In [ ]:
doc = geocoder('Lyon')

* Geocoder une liste de noms de lieux :

In [ ]:
doc = geocoder(['Lyon', 'Annecy', 'Chamonix'])

### 4.3 Visualiser les résultats

* Afficher le résultat GeoJSON :

In [ ]:
print(doc.geojson)

* Afficher la liste des toponymes candidats :

In [ ]:
for t in doc.toponyms: 
    print(f'lat: {t.lat}\tlng: {t.lng}\tsource {t.source}\tsourceName {t.source_name}')

* Récupérer les toponymes candidats sous la forme d'un geodataframe :

In [ ]:
doc.to_geodataframe()

* Afficher la carte des résultats

In [ ]:
doc.get_folium_map()

## 5. Les jeux de données


La libraire [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

### 5.1 Articles encyclopédiques

Le jeu de données des articles encyclopédiques est disponible dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). 

%%Nous allons charger la version brute et voir comment manipuler un dataframe.

#### 5.1.1 Corpus brut

In [ ]:
from perdido.datasets import load_edda_artfl 

dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']
data_artfl.head()

In [ ]:
data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN'].text.item()

#### 5.1.2 Corpus traité par Perdido

In [ ]:
from perdido.datasets import load_edda_perdido

dataset_perdido = load_edda_perdido()
data_perdido = dataset_perdido['data']
df = data_perdido.to_dataframe()
df.head()

In [ ]:
doc = data_perdido[0]
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 


### 5.2 Descriptions de randonnées (traitées par Perdido)

In [ ]:
from perdido.datasets import load_choucas_perdido

dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

df = data_choucas.to_dataframe()
df.head()

In [ ]:
doc = data_choucas[2]

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [ ]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True) 

## 6. Pour aller plus loin

1. Tutoriel (en français) Geoparsing : [https://github.com/ludovicmoncla/tutoriel-geoparsing](https://github.com/ludovicmoncla/tutoriel-geoparsing)
2. Tutoriel (en français) présenté lors de l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org) : [https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing)
3. Tutoriel (en anglais) utilisé pour le cours [SunoikisisDC](https://sunoikisisdc.github.io) Summer 2022 Course on [Natural Language Processing (NLP) for historical texts](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9) (Session 9)